In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

from datetime import datetime
import os

In [2]:
housing = fetch_california_housing()
m, n = housing.data.shape

In [3]:
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [4]:
learning_rate = 0.01
n_epochs = 1000

In [5]:
# Constructing the Graph

# Input Variables and Parameters
with tf.name_scope("Inputs") as scope:
    X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name= 'X')
    y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name= 'y')
    
theta = tf.Variable(tf.random_uniform([n + 1, 1], minval=-1.0, maxval=1.0), name='theta')

# Making predictions
y_pred = tf.matmul(X, theta, name='y_pred')

# Calculating errors
with tf.name_scope("Loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name='mse')

# Back-progation process
''' In these two lines of code, the gradients of the cost function is calculated with respect to all the parameters in the model
as done by tf.gradients(mse, [theta])[0] and the parameters are updated using the delta rule in                                 case of Gradient Descent
with the given learning rate as done by tf.assign(theta, theta - learning_rate * gradients) '''

with tf.name_scope('optimization') as scope:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(mse)


In [6]:
now = datetime.utcnow().strftime("%Y-%m-%d-%H:%M:%S")
root_logdir = "tf_logs"
logdir = os.path.join(os.getcwd(),"{}/run-{}".format(root_logdir, now))

In [7]:
# Creating Summary stats and writing it to file along with the graph definition

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [8]:
init = tf.global_variables_initializer()

sess = tf.InteractiveSession()
sess.run(init)
for epoch in range(n_epochs):
    if epoch % 100 == 0:
        print('Epoch ', epoch, 'MSE :', mse.eval())
        # Adding Summary
        file_writer.add_summary(mse_summary.eval(), epoch)
    sess.run(training_op)

np_mse, np_theta, np_y_pred = sess.run([mse, theta, y_pred])

sess.close()

Epoch  0 MSE : 8.16029
Epoch  100 MSE : 0.642999
Epoch  200 MSE : 0.547652
Epoch  300 MSE : 0.540235
Epoch  400 MSE : 0.535992
Epoch  500 MSE : 0.532923
Epoch  600 MSE : 0.530679
Epoch  700 MSE : 0.529035
Epoch  800 MSE : 0.527828
Epoch  900 MSE : 0.526939


In [9]:
file_writer.close()